<a href="https://colab.research.google.com/github/urosgodnov/juypterNotebooks/blob/main/DataMining/textMining_scraping_data_with_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Permanently install libraries

## Initializing gdrive

In [1]:
# permanently install packageg in google colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Creating links

In [ ]:
import os
nb_path = '/content/drive/MyDrive/Google_Colab_modules'

if not os.path.exists(nb_path):
  os.symlink('/content/drive/MyDrive/Google_Colab_modules', nb_path)  # Create the symlink only if it doesn't exist
else:
  print(f"Path '{nb_path}' already exists. Skipping symlink creation.")


## Installing to destination folder in gdrive

In [ ]:
#install the module in the
!pip install --target=$nb_path selenium --quiet

In [ ]:
!pip install --target=$nb_path webdriver_manager --quiet

## Connecting the path

In the future, while all modules will be installed, we will only use this command.

In [13]:
import sys

sys.path.append("/content/drive/MyDrive/Google_Colab_modules")

# Webscraping

In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    ElementClickInterceptedException,
    TimeoutException
)
from datetime import datetime



In [15]:
import pandas as pd
import time

In [46]:
options = webdriver.ChromeOptions()
options.add_argument("--verbose")
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument("--window-size=1920, 1200")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like \
Gecko) Chrome/131.0.6778.109 Safari/537.36')

In [44]:
# Define the path to selenium-manager
selenium_manager_path = '/content/drive/MyDrive/Google_Colab_modules/selenium/webdriver/common/linux/selenium-manager'

# Grant execute permissions
!chmod +x "{selenium_manager_path}"

In [53]:
driver = webdriver.Chrome(options=options)

In [54]:
l="https://www.imdb.com/title/tt18412256/reviews/?ref_=tt_urv_sm"
driver.get(l)

In [55]:
wait = WebDriverWait(driver, timeout=5)


In [ ]:
i=1
while True:
    try:
        # Locate the "25 More" button
        button = wait.until(EC.presence_of_element_located((By.XPATH, "//span[@class='ipc-see-more__text']")))

        # Click the button using JavaScript
        driver.execute_script("arguments[0].click();", button)
        print('Clicked the "25 More" button', i, ' time(s)!')
        i=i+1
        time.sleep(2)


    except NoSuchElementException:
        print('The "25 More" button no longer exists. Exiting loop.')
        break  # Exit the loop when the button is no longer found

    except Exception as e:
        print('An unexpected error occurred')
        break  # Exit the loop for other unexpected errors




In [57]:
from bs4 import BeautifulSoup
content = driver.page_source
soup = BeautifulSoup(content)
result=[]
df = pd.DataFrame(columns=["Author","Title","Rating","Text","Date"])

In [58]:
def GetAReview(review):
  author = rev.findAll('a', attrs={'class':'ipc-link ipc-link--base'})[0].text
  title = rev.find('h3', attrs={'class':'ipc-title__text'}).text.strip()
  rating =rev.findAll('span', attrs={'class':'ipc-rating-star--rating'})[0].text.replace("\n","").strip()
  review = rev.findAll('div', attrs={'class':'ipc-html-content-inner-div'})[0].text
  date = rev.findAll('li', attrs={'class':'ipc-inline-list__item review-date'})[0].text
  date=datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')

  return pd.DataFrame({"Author":author, "Title":title, "Rating":rating, "Text":review, "Date":date},index=[0])


In [59]:
for rev in soup.findAll('article'):
  try:
    result.append(GetAReview(rev))
  except:
    continue

In [ ]:
df=pd.concat(result, axis=0, ignore_index=True)
df.info()

In [ ]:
df.head()

In [62]:
df.to_csv("Alien.csv")